<a href="https://colab.research.google.com/github/samancha/nlp-master/blob/main/mod5/NLP_mod5_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data loading and preprocessing.

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
! pip install transformers

In [63]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch
import nltk
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.metrics import classification_report
# Download the stopwords and tokenizer from nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# specify GPU
device = torch.device("cuda")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [64]:
def preprocess_text(text):
    # Convert text to lowercase
    words = word_tokenize(text)

    # Convert words to lowercase
    words = [word.lower() for word in words]

    # Remove punctuation from words
    words = [word for word in words if word.isalnum()]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Remove duplicate words
    unique_words = list(dict.fromkeys(words))

    # Join the words back into a string
    text = ' '.join(unique_words)

    return text


In [65]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/archive.zip', )
df['label'] = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
df['processed_review'] = df['review'].apply(preprocess_text)
df.reset_index(drop=True)
display(df.head())

,review,sentiment,label,processed_review
0,One of the other reviewers has mentioned that ...,positive,1,one reviewers mentioned watching 1 oz episode ...
1,A wonderful little production. <br /><br />The...,positive,1,wonderful little production br filming techniq...
2,I thought this was a wonderful way to spend ti...,positive,1,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,negative,0,basically family little boy jake thinks zombie...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1,petter mattei love time money visually stunnin...


In [32]:
# train_df, test_df =
X_train, X_test, y_train, y_test = train_test_split(df['processed_review'], df['label'], test_size=0.2)

# Text tokenization and conversion to BERT input features.

In [66]:
# Load the pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

REF: https://huggingface.co/docs/transformers/pad_truncation

In [ ]:
inputs = df.processed_review.values
labels = df.label.values
print("Train data size ", len(inputs))
print(' Original: ', inputs[0])
# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(inputs[0]))
# Print the sentence mapped to token ids.
print('Token IDs: ', bert_tokenizer.convert_tokens_to_ids(bert_tokenizer.tokenize(inputs[0])))

In [69]:
# Tokenize and convert training data to input features
train_encodings = tokenizer(train_reviews, truncation=True, padding=True)
train_input_ids = train_encodings["input_ids"]
train_attention_mask = train_encodings["attention_mask"]

ValueError: ignored

In [59]:
print(type(train_encodings))
print(type(train_input_ids), len(train_input_ids) )
print(len(y_train))

# Create DataLoader
train_dataset = TensorDataset(train_encodings.input_ids, train_encodings.attention_mask, torch.tensor(list(y_train)))
# test_dataset = TensorDataset(test_encodings.input_ids, test_encodings.attention_mask, torch.tensor(list(test_df['sentiment'])))
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# # Initialize optimizer
# optimizer = AdamW(model.parameters(), lr=1e-5)

<class 'transformers.tokenization_utils_base.BatchEncoding'>
<class 'list'> 40000
40000


AttributeError: ignored

In [ ]:
# Tokenize and convert testing data to input features
test_encodings = tokenizer(test_reviews, truncation=True, padding=True)
test_input_ids = test_encodings["input_ids"]
test_attention_mask = test_encodings["attention_mask"]

In [54]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

labels = [1,0]

# For every sentence...
for sent in X_train:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        # max_length = 64,           # Pad & truncate all sentences.
                        padding=True,
                        truncation=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', X_train[0])
print('Token IDs:', input_ids[0])

RuntimeError: ignored

In [26]:
display(attention_masks)
# labels = torch.tensor(['1','0'])

tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])

ValueError: ignored

# Model definition, training, and evaluation.

#### Sequence Classifacation

In [18]:
# Load BertForSequenceClassification, the pretrained BERT model with a single
# linear classification layer on top.
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
# Tell pytorch to run this model on the GPU.
model.cuda()

TypeError: ignored

In [10]:
# tf_dataset = model.prepare_tf_dataset(train_reviews, batch_size=16, shuffle=True, tokenizer=tokenizer)


AttributeError: ignored

### Fine-tune the BERT model on the preprocessed IMDb dataset for sentiment analysis.

### Implement training loops and loss calculation.

In [19]:
batch_size = 2
epochs = 3
optimizer = AdamW(model.parameters(), lr=2e-5)
model.train()

for epoch in range(epochs):
    for i in range(0, input_ids.size(0), batch_size):
        batch_input_ids = input_ids[i:i+batch_size]
        batch_attention_masks = attention_masks[i:i+batch_size]
        batch_labels = labels[i:i+batch_size]

        optimizer.zero_grad()

        outputs = model(
            input_ids=batch_input_ids,
            attention_mask=batch_attention_masks,
            labels=batch_labels
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ValueError: ignored

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


### Evaluation

testing set using accuracy, precision, recall, and F1-score metrics

# Sample movie review predictions and explanations.


In [55]:
# Perform inference
with torch.no_grad():
    outputs = model(input_ids[0], attention_mask=attention_mask[0]).logits

# Get predicted label
predicted_label = torch.argmax(outputs, dim=1).item()

# Define label names
label_names = [1, 0]

# Print result
print("Text:", input_ids[0])
print("Predicted Label:", label_names[predicted_label])

AttributeError: ignored